<a href="https://colab.research.google.com/github/wagh23/isi_cryptography_internship_2019/blob/first/kasiskiMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cryptanalysis of Vigenere Cipher

The following is the cryptanalysis of the ciphertext encrypted uses the vigenere cipher given in this [problem statement](https://www2.isical.ac.in/~rcbose/internship/problems2019.pdf), using the Kasiski Method




In [ ]:
from collections import Counter
import string
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import re
from collections import defaultdict



# Ensure the words corpus is downloaded
nltk.download('words')
nltk.download('punkt')


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Guessing Keyword Length

We will first find out the trigrams and their distances, and since for such a large ciphertext there maybe accidental coincidences resulting in substring pairs that are the same but the plaintext or key substring isn't. For this reason we consider keylengths up to 20 and see how many distances a specific possible keylength is a factor of.


In [ ]:
def factor_frequencies(distances, max_factor=20):
    freq = {}
    for f in range(1, max_factor + 1):
        freq[f] = sum(1 for d in distances if d % f == 0)
    return freq
def get_trigram_distances(ciphertext):
    ciphertext = ''.join(filter(str.isalpha, ciphertext.upper()))
    trigram_positions = defaultdict(list)

    # Find all trigram positions
    for i in range(len(ciphertext) - 2):
        trigram = ciphertext[i:i+3]
        trigram_positions[trigram].append(i)

    distances = []
    for positions in trigram_positions.values():
        if len(positions) > 1:
            for i in range(1, len(positions)):
                distances.append(positions[i] - positions[i - 1])

    return distances
def find_common_factors(distances):
    def all_factors(n):
        return set(f for i in range(1, int(n**0.5)+1) if n % i == 0 for f in (i, n//i))

    common_factors = None
    for d in distances:
        factors = all_factors(d)
        if common_factors is None:
            common_factors = factors
        else:
            common_factors &= factors

    return sorted(common_factors) if common_factors else []

# Example usage after getting distances
ciphertext = "FBHJLZINOHBYTYCNCZLUYZUFIGDMZMYKVQDSPUEFJIGJPOQUFQDRJQJNTVDREXXJEOQGDUUWFWGOXQQHKJLUWFDYCMFRZTUIKVHHZPATEHKKLHUTWRLTZTQZIGEOCEIFISWNPPDQPAHSMFHXFTWNPEYSFTDSTMOYISHZSBJXLFYOGFTYYSRXOFQQRBGZSFEUVBQONIUXCSIZMFXNERJGGFJMVADTOPKWVOURJNQRDOOGYDUXKCUYEIUNIHLSPJDYYSHIZMELZQDRDQEYCWJNECKYNVDZTGSFCOPOEZXFUBWHPGQQCSQZSFTNECVGFSIBFIOJEIUDJHLRWIQAVURTPPKYECWCTUXFSOQMMVJFNVLSAFHRRMEKYPJFTQRXOJDLKCDTPXIYLRBZSBJXRMVJTOEXRIUYDUYQCVDJAMUSKMRLGJCFERYORPHQVOGOYHKUKCWNPNQXJSAZTOSYZCQGEUXJVBGUQUXJTFHZLDUTLGSKCJEIISYKLMUILGLTRIKLVGLSFMQYZCQYEIQYRFHTPXJTGOOKZOJTCCJEEIUKZBGOYHCFIYVZSFBFKSVZEVHSZBDJPCQYVCYKCXXJKVHXOJDTJOXXDXUWVOOXPBTDZBWKCNYSRZGKNMYSVPBZSFJNDSGUZNIIRMVZCVSPZBDJOJJNFBWNPTJZUMVIFUJNEUHJRFQUGFRGNISTLZGNPMFZJPHZEFHQFCNHLDAFKDDYEFDAZFRTXFDYRZWACNENCOQJWFQWEWQLTOUWUSWGTMMMRHZKXJWMKSAVPDJKICPSZEUWEQOOXBJJTVDTRFVTIAHGYEVTIOOUEPVUVCSRPUXFKKLYSUXNJHKKEFQRYOVKYEUIWCUSPBTJTWVOZOCFUSLZDFURVRDYTGXJYOGHPFDUCOBOYHMNKVXYLMBMZGOOQFYKVSOYZSHDWCUZSPIJNVRCLOJYFQRSAFJJWCUSPTINJHKXZOUNKGLSAPIXZPOKEIYXBWGODVDNHIHSPTINZGDTLMYJEHKGEEUIZQDZPTXNDGHRQUEUCOBOYHMNKVKAXBDXZVDBPGKSCWNKLDXNCRLTEIUXKFHKEXXJEHKKOBOHFAHYHIUSZAQUEFDOFMLTRJJNNWORWFQAVTRUECQQCAHYDJYXKVHHPTJUCOBKCJDYYSZUCMTGPGRSPEYXKOQIPIUXCWNKLQBFPGWGEJESYSFGYUQPVOGBLOJFXSRLPWUWPALYEBAJNSPGVFJMRHLYXFQSUAHYDJMJISUORIJZGHKKCFQGJCOAEFBDCSWSPFCGIOFKEIUJJCXXLELJIGLZJGEWNWVKXFDXRMLZTTJMVKLYPTJHFIUYPJWSFFDTNFYXKVHIFSIJFTJUOLDTNZHJRFYXKVHCTOWBYSUKHJJMNSIRJUEMVOYKYUXNJZLLPXXNTVKGOCUJEHKKEPCGFTKODWYWKIHGYEEKYWVNZOEZIWVHFUQBRZNOYHIMRRRCLQETIDOGJFHYYOWYESKYJOQJQSUYJVLYSPKWLDRTEIUXKOJKLOTYYSQODIUFIRQUXPHJZHLYLUQQVHRROCOFEWGOZUVZCZRLDPKSUOQJQVHDJWJTTGONEUQUEIYSXOIZPSJMVPLMMBDLZHZGDDEQUOQJMMQHBOQJEIUSKVHXPXQXCWJNEOEBWCUZSFVNIGWZTNUFJHUUYPCJIGKGGFWQZASYPEJMRHGGHOEKKVHAYJLJIGHZSJHYVSQHTMBNFBBKLSIFXCZNPOJMVSDXWJUXKGWGCTMJISWACOYSXCQZSFBNXVWOYUXJTCVSTDTFIYQKDTQSUWIZSBJNJBRZPOEZXVWNPZCFPVDBPEUYVQWKONOXKSUOZVIIRFNSLUJJIOWCZSAYFCWNPHBNDDVKNPDXZGWKOPVFWOLTESQIZCVOROQQWFRSOFUUJDDIPQYHBSGAACOFEOQZPODFKVDZTTIQZUKZWZRNXUHXEIQSRFHLCJWJIOWUCBDITCVZDMUXJHKGYGYAVALRWJESSIWOYDUWKOLTHBOXTOQMZCQHBAXISGQWKVHXTOJNDSDTOEYXKOQIPUXFEHKKNFBJSFDZPECZCHLHTMBNFBGUWMQWYIEHWFIURQHZPMUXTCSKUVTISCZSLOEKRFLFZOQXKOWKFOYAVFVOEZBJRRDAEIEWFTDYEVTDZBMUFSDFCBDZFSUXRWGZSFINXBDRNBCJWFRSEIUAVFBLTSIYFPMKNUINEHKKFOYAVFVKLTYYNOVKXFHLZBJUFUEKUOUQYFIXKKRNFOTWVRPOWMYTEMHGCTQKKSUZSFRNXPDTRTUJZBJZSFKSZJHXDFZZJHOORIJNEUXVPWUSKVRARIYYNOVUYMOFWOLTETYLEOOODFLJEARXPJCUFFWGYUJMRBWNPCYLSOQMMFSFLGHCPBHJDOGKZGIYRFVZFGVFERVUHFQWVUOOXQINEUDZZVHTIWJOYTQNUOVZCPDTDSUXTDXFIRHRWJIBYCZGDOEYZBYUWWUIZBWNPQHTASFZEIUXZUQGWTXTNSGAYFNUVQWKOMOHFZGZPNFJIOWACFIFERDTFOKXLOORJQHTECXTNFTBRJHCSFDFJHUUYPCJIGWXTFTYFTLMFSUTLHZNJUXJSSVZPYFQRBDZTPDBRGWNLUUQLGLBPEQWBADZEFHRRMKGGFRJVBDZHPHPZTYKCJVNVRWNLUMTLZGHPUXJWWUYEDESWWUSLUYTECIOETANERRLOBHPDOWZPSMMZQKODBIZSGWGYUYFCDDXEPVYYSXTTWUWJSWNLUIHZSQZTTJXYOYKMFUSJSDXNIYSXTRXZWUWUSFGOFINWQRTQJHRVRWNTTTNJQRBPSOIVGHXGFIYNCQUMFBUIWCKDGEWSCWNNBFYLFLTRUXJJWJTLMEKKVHLTSIYJHDXDBDIGCWKYUYFCRDXVNQYKSUIZOVNIADZTPDXRWGNLSLFIRDYESESFAHXLWYQFSECSPMFJBRZABHYFTWNPSUXVOUISUUFDQDAEJESZBJZSBJJOHUGZSTNEOUENMQNDGUKBVYWVSAZCBEWUWQGCZUAZRHTNFXJJOLJTOTJGSQJPOJYVGWYLSUSVSGKOUEAVFLLJUXJWWQJTOWXSCZSLOQLISHJTOTJGSQJPOJYVGWYLSUSVSGKOFLJEHKUFHXMZGWKLNIUVBWZHPOJRFVJZVRQVOQJESYUCSFNPDANEUWNPJHBFFNOEJIFKWPKZGJMVIQOGFHXVKHXPBBQPRRTZUASFKDTJUXNEUDHZVJGFKPGYTQNUVHYLJTYYSGODDEAVFBODMYPVHKKQJHXKGHTEFDHVWQGYFQWCMFNLQJJICIZSFXNJHRXJPVYYSFUDNEXKVLYTTDTKVLTRUXFKOVZCPDTDSUYNPKQUOFZFBBQPGHKTOVFTHLZTTQQCWQJTSUHKPDYPEESTVDTRFINEHKKHBLJCSQMEIIUICGANFTGPFDJTPINXBDRDUXJVOURJVDNMSUYPXQXUOUQLOTHFZGLTMBJUKLZSKKXKVBJCPWJEOQJSFBNLARTNFIYRFVLZSCJUHKKJFCNKHHJFMJWRJLUWFJQZUKZTOJTKVHJLSAFISDYMFJBVSQZSFCYYOWAWUHFMWRRPUBNXVWISBDLVGWNPFDJIUBYTHDFKIUKZGXDUFRMPOQYFAVHZXCFEGDOOBIYICQUXFHXCCRQPEQYRGSKNJVNTKDBPMUSXHKOQUXJISZKCFIYRFVGYEKQKFDBTPBJKZLMSUJMVMZUFMTXVSRTPTYLEOWACFYKKVHXPXUWVBRYEBHXKVHEHPKQUGHKLOEYYSUZSFOXRKDIWFQWSIWLLJDYJWJTLMIMFKLTRUXJISZKCFIYRFVVCPRFSZBSLOOTWHKKXCEBDOQYLJTKZBGOYHJMRHWXLDUXZUQGWXQXECWKLTOGVQDADFJMVALRVZMFPUDRLYOFCCQKMPERJKLZSSQIZCZGGFDTZGHZPOJMFIVGYEJNDSVRZVTJIGDOOQUYVFNACDPDEGNOLELFEQHJASELIOPZPDXSFZRMJEYWVQWUCGEWKVHTLUYTEOOYNJUSTSIUFOTFKWRTHIYHYVHRAFTKLBGZSFHJJSDXNIVNERLTRUXJZASGNUEKKVHLTSIYJHDXDJDYYOWILDEUYCQEHPKQUPHRTLUYIMLTRUEMVOUZSFVQRDRLLIKRDWQMMJHIJKLTRGHTDWQYTEUFYIUXTDQSVYXXNAOSJYLYLJTNEOQTDGLNUSRHPDQZJSWNPIYLYSQJZGJMVTUKBVUSTMWNPZMJISOUZLYSXWQODUXJJOPKLTVRIOGOZUXJRGWXZOERVFVNLEJTXCWUEIUFLGWXLMYFERHYPSJYFSVILQUNEHHXQFHJEQHZSBJBRGZNPSUYYSBOYTJFCZHJEIUNIOQZPODFJHKKJUXJEZDHZSUIKCFUYGYWDKKGEUXJPTRAYEYSGOUZMZJJJHLTRJJFXOLTDUTZDABYTHDFCGLTEIUQRPDTOJJFCZVNZXUIKVDZHIQYKVHEDQEYKSGCLTJMVSAODUUSTSRLEIUKZFVZDUQWJPRCXBDXRWG"
distances = get_trigram_distances(ciphertext)
freqs = factor_frequencies(distances)

print("Trigram Distances:", distances)
print("Factor Frequencies (1 to 15):")
for factor, count in freqs.items():
    print(f"{factor}: {count}")
# print(len(ciphertext))

Trigram Distances: [2388, 3552, 3600, 832, 2674, 1328, 105, 3701, 1504, 3936, 888, 3561, 3856, 1062, 2487, 1255, 376, 32, 872, 1384, 3008, 56, 3584, 3638, 281, 216, 2496, 1352, 744, 2600, 1016, 528, 488, 1016, 3304, 3432, 1856, 1576, 704, 4432, 13, 3456, 2289, 912, 568, 2344, 3456, 3456, 648, 3208, 4296, 80, 4376, 496, 544, 208, 24, 264, 64, 328, 848, 680, 192, 16, 256, 72, 392, 1040, 1991, 1848, 904, 72, 336, 632, 160, 920, 280, 736, 240, 176, 160, 632, 160, 1064, 136, 112, 320, 536, 726, 952, 3008, 3432, 1624, 2624, 1464, 16, 304, 216, 1144, 48, 512, 112, 480, 216, 424, 616, 24, 4079, 2991, 73, 448, 2816, 2816, 1016, 4264, 1042, 2128, 157, 1992, 2294, 904, 4084, 2936, 1544, 1053, 1579, 49, 232, 1400, 1192, 1352, 232, 1400, 1192, 1352, 232, 718, 3032, 1251, 2408, 1344, 703, 1033, 80, 896, 368, 1224, 512, 64, 456, 576, 446, 200, 2424, 210, 3872, 1984, 825, 1063, 120, 256, 72, 1016, 112, 408, 104, 16, 1016, 264, 208, 280, 80, 2952, 2945, 872, 104, 336, 984, 1684, 976, 57, 3698, 3104, 30

It is evident that the keylength is likely 4 or 8 (note that 16 is also a very common factor)

To be sure we will find out the Indices of Conicidence for numbers up to 15 (since there is a decline in factor frequency)

In [ ]:
def index_of_coincidence(text):
    N = len(text)
    if N <= 1:
        return 0.0
    freq = [text.count(chr(i)) for i in range(ord('A'), ord('Z') + 1)]
    ic = sum(f * (f - 1) for f in freq) / (N * (N - 1)) #formula for I_c
    return ic

def average_ic_for_keylength(ciphertext, keylength):
    ciphertext = ''.join(filter(str.isalpha, ciphertext.upper()))
    substrings = ['' for _ in range(keylength)]

    for i, char in enumerate(ciphertext):
        substrings[i % keylength] += char

    ic_values = [index_of_coincidence(sub) for sub in substrings]
    avg_ic = sum(ic_values) / keylength
    return avg_ic

def run_ic_scan(ciphertext, max_keylength=15):
    ciphertext = ''.join(filter(str.isalpha, ciphertext.upper()))
    print("Key Length | Avg Index of Coincidence")
    print("-------------------------------------")
    for keylen in range(1, max_keylength + 1):
        ic = average_ic_for_keylength(ciphertext, keylen)
        print(f"{keylen:10} | {ic:.4f}")

# Example usage
run_ic_scan(ciphertext)


Key Length | Avg Index of Coincidence
-------------------------------------
         1 | 0.0405
         2 | 0.0449
         3 | 0.0404
         4 | 0.0538
         5 | 0.0404
         6 | 0.0448
         7 | 0.0402
         8 | 0.0662
         9 | 0.0402
        10 | 0.0450
        11 | 0.0403
        12 | 0.0537
        13 | 0.0404
        14 | 0.0444
        15 | 0.0405


It is clear that 8 is probably our keword length

# Frequency Analysis

In [ ]:

# English letter frequency in percentage (from most to least common)
ENGLISH_FREQ = {
    'E': 12.7, 'T': 9.1, 'A': 8.2, 'O': 7.5, 'I': 7.0,
    'N': 6.7, 'S': 6.3, 'H': 6.1, 'R': 6.0, 'D': 4.3,
    'L': 4.0, 'C': 2.8, 'U': 2.8, 'M': 2.4, 'W': 2.4,
    'F': 2.2, 'G': 2.0, 'Y': 2.0, 'P': 1.9, 'B': 1.5,
    'V': 1.0, 'K': 0.8, 'J': 0.15, 'X': 0.15, 'Q': 0.1, 'Z': 0.07
}

def get_letter_frequencies(text):
    counter = Counter(text)
    total = sum(counter.values())
    return {char: (counter[char] / total) * 100 for char in string.ascii_uppercase}

def shift_text(text, shift):
    return ''.join(chr((ord(c) - ord('A') - shift) % 26 + ord('A')) for c in text)

def chi_squared_score(text_freq):
    return sum(
        ((text_freq.get(ch, 0) - ENGLISH_FREQ[ch]) ** 2) / ENGLISH_FREQ[ch]
        for ch in ENGLISH_FREQ
    )

def analyze_key_length(ciphertext, key_len):
    ciphertext = ''.join(filter(str.isalpha, ciphertext.upper()))
    groups = ['' for _ in range(key_len)]

    # Split ciphertext into key_len groups
    for i, char in enumerate(ciphertext):
        groups[i % key_len] += char

    key = ''
    for i, group in enumerate(groups):
        min_chi = float('inf')
        best_shift = None
        for shift in range(26):
            shifted = shift_text(group, shift)
            freq = get_letter_frequencies(shifted)
            chi = chi_squared_score(freq)
            if chi < min_chi:
                min_chi = chi
                best_shift = shift
        key_letter = chr((best_shift % 26) + ord('A'))
        key += key_letter
        print(f"Group {i+1}: Best shift = {best_shift}, Key letter = {key_letter}")
    print(f"\n Estimated key: {key}")

# Example usage
key_length = 8
analyze_key_length(ciphertext, key_length)


Group 1: Best shift = 17, Key letter = R
Group 2: Best shift = 14, Key letter = O
Group 3: Best shift = 3, Key letter = D
Group 4: Best shift = 6, Key letter = G
Group 5: Best shift = 11, Key letter = L
Group 6: Best shift = 1, Key letter = B
Group 7: Best shift = 16, Key letter = Q
Group 8: Best shift = 5, Key letter = F

 Estimated key: RODGLBQF


# Decryption

In [ ]:
def vigenere_decrypt(ciphertext, key):
    ciphertext = ''.join(filter(str.isalpha, ciphertext.upper()))
    key = key.upper()
    key_len = len(key)

    decrypted = ''
    for i, char in enumerate(ciphertext):
        shift = ord(key[i % key_len]) - ord('A')
        decrypted_char = chr((ord(char) - ord('A') - shift) % 26 + ord('A'))
        decrypted += decrypted_char

    return decrypted

# Example usage:
key = "RODGLBQF"
plaintext = vigenere_decrypt(ciphertext, key)
print(plaintext)

ONEDAYSIXTYSIXMILLIONYEARSAGOLIFECAMETOASUDDENAPOCALYPTICHALTWHENANASTEROIDIMPACTVIOLENTLYCLOSEDTHEBOOKONTHEAGEOFDINOSAURSBIRDSARETHEONLYMEMBERSOFTHEDINOFAMILYTREETHATSURVIVEDTHEORDEALANDTHEOPENNICHESLEFTBEHINDGAVETHEMANDOUREARLYMAMMALANCESTORSTHEIRTIMEINTHEECOLOGICALSPOTLIGHTBUTWHATIFCALAMITYHADNTBEFALLENTHEDINOSAURSWOULDTHEYSTILLHAVEGONEOUTNOTWITHABANGBUTAWHIMPERMAYBENOTACCORDINGTOANEWSTUDYTHATSAYSDINOSAURSSTILLHADPLENTYOFVIMANDVIGORLEADINGUPTOTHEMASSEXTINCTIONATTHEENDOFTHECRETACEOUSPERIODREVEALEDUSINGHUGESIMULATIONSTHATARENEWTOPALEONTOLOGYTHEFINDINGMARKSTHELATESTTURNINADEBATEOVERWHETHERDINOSAURSWEREALREADYINTERMINALDECLINEBYTHETIMEDOOMSDAYSTRUCKINADDITIONTHESTUDYSCUTTINGEDGEAPPROACHCOULDHELPUSBETTERLOOKBACKATPASTENVIRONMENTALTURMOILANDLEARNINFINERDETAILWHATWEMIGHTEXPECTFROMMODERNCLIMATECHANGEFORMEANDFORALOTOFPEOPLETHATWISHTHISTHETEAMHASENDEDFORMEADECISIONMADEITSEEMEDASIFHEHADBEENPLAYINGWITHUSALLHISLIFEIFEELSORRYFORTHOSEWHOWANTTOCOMPETEFORMESSISTHRONEITSIMPOSSIBLETHISKIDISUNIQUEMESSI

# Attempt at adding whitespace

In [ ]:
#Load English words
word_list = set(words.words())
text = plaintext

# Simple dynamic programming word breaker
def word_break(text, word_dict):
    n = len(text)
    dp = [None] * (n + 1)
    dp[0] = []

    for i in range(1, n + 1):
        for j in range(i):
            if dp[j] is not None and text[j:i].lower() in word_dict:
                dp[i] = dp[j] + [text[j:i]]
                break
    return dp[n]

# Try to reconstruct the text with spacing
broken_text = word_break(text, word_list)

# Join into a string
if broken_text:
    spaced_text = ' '.join(broken_text)
else:
    spaced_text = "Could not segment text properly."

spaced_text[:1000]  # Show a preview


'ONE DAY SIXTY SIX MILLION YEAR SAGO LIFE CAME TO ASUDDEN APOCALYPTIC HALT W HE NAN ASTEROID IMPACT VIOLENTLY CLOSED THE BOO KON THE AGE OF DINOSAUR S BIRD SA RETHE ONLY MEM B ERS OF THE DI NO FAMILY TREE THAT SURVIVE D THE OR DE ALAND THE OPEN N ICH ES LEFT BEHIND GAVE THE MAN DOUR EARLY MAMMAL ANCESTOR S THEIR TI MEIN THE ECOLOGICAL SPOTLIGHT BUT WHAT IF CALAMITY HAD N T BE FALLEN THE DINOSAUR S WOULD THEY STILL HAVE GONE OUT NOT WI THA BANG B UTA WHIMPER MAYBE NOT ACCORDING TO ANEW STUDY THAT SAY S DINOSAUR S STILL HAD PLENTY OF V I MAND VIGOR LEADIN GUP TO THE MASS EXTINCT IO NAT THE END OF THE CRETACEOUS PERIOD REVEALED U SING HUGE SI M ULA TI ONS THA TA RENEW TO PALEONTOLOGY THE FINDING MARK S THE LATEST TUR NI NA DEBATE OVER WHETHER DINOSAUR S WE REAL READ YIN TERMINAL DECLINE BY THE TIME DOOMSDAY ST R U C KIN ADDITION THE STUDY S CUTTING EDGE APPROACH COULD H EL PUS BETTER LOOK BAC KAT PAST ENVIRONMENTAL TUR MO I LAND LEARN IN FINER DETAIL WHAT WE MIGHT EXPECT FROM MODERN CLIMA